In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
!pip install spacy --upgrade

Requirement already up-to-date: spacy in /usr/local/lib/python3.7/dist-packages (3.0.6)


In [ ]:
%%bash
python -m spacy download en
python -m spacy download de

  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ As of spaCy v3.0, shortcuts like 'de' are deprecated. Please use the
full pipeline package name 'de_core_news_sm' instead.
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


2021-06-21 17:14:51.319173: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-06-21 17:14:57.779417: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [ ]:
import pandas as pd 
dataset = []
s08 = "/content/drive/MyDrive/Question_Answer_Dataset_v1.2/S08"
s09 = "/content/drive/MyDrive/Question_Answer_Dataset_v1.2/S09"
s10 = "/content/drive/MyDrive/Question_Answer_Dataset_v1.2/S10"

data_set_folders = [s08,s09,s10]
def prepare_dataset(folders_list):
  question_answer_pair = "question_answer_pairs.txt"
  final_data_set = []
  for folder in folders_list:
    abs_qa_path = folder + "/" + question_answer_pair
    year_data_set = generate_single_year(abs_qa_path,folder)
    print(len(year_data_set))
    final_data_set += year_data_set
  return final_data_set
    
def generate_single_year(filename,year_path):
  year_data_set = []
  with open(filename,'r',encoding="ISO-8859-1") as f:
    for line in f.readlines():
      cleaned_line = line.strip().split('\t')
      year_data_set.append(cleaned_line)
  return year_data_set

final_dataset = prepare_dataset(data_set_folders)
headers = final_dataset[0]

1716
826
1459


In [ ]:
len(final_dataset)

4001

In [ ]:
dict(enumerate(headers))
df = pd.DataFrame(final_dataset)[1:]
df.rename(columns=dict(enumerate(headers)),inplace=True)
df.drop(columns=["ArticleTitle","DifficultyFromQuestioner","DifficultyFromAnswerer","ArticleFile"],inplace=True)
df.head()

,Question,Answer
1,Was Abraham Lincoln the sixteenth President of...,yes
2,Was Abraham Lincoln the sixteenth President of...,Yes.
3,Did Lincoln sign the National Banking Act of 1...,yes
4,Did Lincoln sign the National Banking Act of 1...,Yes.
5,Did his mother die of pneumonia?,no


In [ ]:
spacy_en = spacy.load('en_core_web_sm')
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

SRC = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [ ]:
import math as m
def get_index(train_perc,val_prec,test_prec,len_data):
  return (int(train_perc*len_data),int(val_prec*len_data),int(test_prec*test_prec))


len_data = df.shape[0]
test_split,val_split,train_split = get_index(0.8,0.1,0.1,len_data)
train,val,test = df.loc[:test_split], df.loc[test_split:test_split+val_split],df.loc[test_split+val_split:]
train.shape, test.shape, val.shape
print(train.shape)
print(test.shape)
print(val.shape)
train_list = train.values.tolist()
test_list = test.values.tolist() 
val_list = test.values.tolist()

(3200, 2)
(401, 2)
(401, 2)


In [ ]:
fields = [('Question', SRC),('Answer',TRG)]

In [ ]:
import random
import torch, torchtext.legacy
from torchtext.legacy import data

example1 = [data.Example.fromlist([train_list[i][0], train_list[i][1]], fields) for i in range(len(train_list))]
example2 = [data.Example.fromlist([test_list[i][0], test_list[i][1] ], fields) for i in range(len(test_list))]
example3 = [data.Example.fromlist([val_list[i][0], val_list[i][1] ], fields) for i in range(len(val_list))]

In [ ]:
train_data = data.Dataset(example1, fields)
test_data = data.Dataset(example2, fields)
valid_data = data.Dataset(example3, fields)

In [ ]:
print(vars(train_set.examples[0]))

{'Question': ['was', 'abraham', 'lincoln', 'the', 'sixteenth', 'president', 'of', 'the', 'united', 'states', '?'], 'Answer': ['yes']}


In [ ]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [ ]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 2583
Unique tokens in target (en) vocabulary: 1519


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    sort_key= lambda x:len(x.Question),
    sort_within_batch=True,
    device = device)


In [ ]:
next(iter(valid_iterator))


[torchtext.legacy.data.batch.Batch of size 64]
	[.Question]:[torch.cuda.LongTensor of size 9x64 (GPU 0)]
	[.Answer]:[torch.cuda.LongTensor of size 47x64 (GPU 0)]

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(2583, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(1519, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=1519, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 9,185,775 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        src = batch.Question
        trg = batch.Answer
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):
            src = batch.Question
            trg = batch.Answer

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 3s
	Train Loss: 4.084 | Train PPL:  59.407
	 Val. Loss: 3.514 |  Val. PPL:  33.595
Epoch: 02 | Time: 0m 3s
	Train Loss: 3.991 | Train PPL:  54.120
	 Val. Loss: 3.492 |  Val. PPL:  32.853
Epoch: 03 | Time: 0m 3s
	Train Loss: 3.917 | Train PPL:  50.271
	 Val. Loss: 3.505 |  Val. PPL:  33.294
Epoch: 04 | Time: 0m 3s
	Train Loss: 3.887 | Train PPL:  48.786
	 Val. Loss: 3.537 |  Val. PPL:  34.347
Epoch: 05 | Time: 0m 3s
	Train Loss: 3.805 | Train PPL:  44.947
	 Val. Loss: 3.540 |  Val. PPL:  34.480
Epoch: 06 | Time: 0m 3s
	Train Loss: 3.729 | Train PPL:  41.631
	 Val. Loss: 3.552 |  Val. PPL:  34.893
Epoch: 07 | Time: 0m 3s
	Train Loss: 3.665 | Train PPL:  39.044
	 Val. Loss: 3.506 |  Val. PPL:  33.301
Epoch: 08 | Time: 0m 3s
	Train Loss: 3.559 | Train PPL:  35.126
	 Val. Loss: 3.535 |  Val. PPL:  34.292
Epoch: 09 | Time: 0m 3s
	Train Loss: 3.512 | Train PPL:  33.505
	 Val. Loss: 3.491 |  Val. PPL:  32.809
Epoch: 10 | Time: 0m 3s
	Train Loss: 3.379 | Train PPL:  29.347


In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.491 | Test PPL:  32.809 |
